In [133]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import re
import os
import pickle
import warnings
import wget
import sklearn
import random
import nltk
import spacy
import textblob
import gensim 
import Sastrawi
from gensim.models import Word2Vec
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, GRU, SimpleRNN
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils.data_utils import pad_sequences
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, recall_score, precision_score
from nltk.corpus import stopwords
from nltk.tree import Tree
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from collections import Counter, OrderedDict
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from spacy.util import minibatch, compounding
from spacy import load, displacy
from spacy.training.example import Example
from nlp_id import stopword
from unidecode import unidecode
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Hammam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Hammam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [134]:
df = pd.read_csv('./tweet.csv')

In [135]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1815 entries, 0 to 1814
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  1815 non-null   int64 
 1   sentimen    1815 non-null   object
 2   tweet       1815 non-null   object
dtypes: int64(1), object(2)
memory usage: 42.7+ KB


,Unnamed: 0,sentimen,tweet
0,0,negatif,Kata @prabowo Indonesia tidak dihargai bangsa ...
1,1,netral,"Batuan Langka, Tasbih Jokowi Hadiah dari Habib..."
2,2,netral,"Di era Jokowi, ekonomi Indonesia semakin baik...."
3,3,positif,"Bagi Sumatera Selatan, Asian Games berdampak p..."
4,4,negatif,Negara kita ngutang buat bngun infrastruktur y...


In [136]:
tweet = df['tweet']
display(tweet[0])
display(tweet)

'Kata @prabowo Indonesia tidak dihargai bangsa asing!   Berita ini ðŸ‘‡ pasti hoax buatan penguasa, ya kan @rockygerung?ðŸ˜œ https://twitter.com/mediaindonesia/status/1117575436337160192?s=21Â\xa0â€¦'

0       Kata @prabowo Indonesia tidak dihargai bangsa ...
1       Batuan Langka, Tasbih Jokowi Hadiah dari Habib...
2       Di era Jokowi, ekonomi Indonesia semakin baik....
3       Bagi Sumatera Selatan, Asian Games berdampak p...
4       Negara kita ngutang buat bngun infrastruktur y...
                              ...                        
1810    Negarawan sejati sll bangga dan mengedepankan ...
1811    1. HRS ceramah di Damai Indonesiaku 2. Perekon...
1812    Mari bangun bangsa dgn mendukung perekonomian ...
1813    Bantu majukan perekonomian bangsa bersama Pak ...
1814    Pak @jokowi mengubah cara pandang ekonomi. Kin...
Name: tweet, Length: 1815, dtype: object

## Text Cleaning

In [137]:
tweet_cleaned = []

  # Looping setiap kalimat
for sentence in tweet:
    # Hapus hashtag
    sentence = re.sub(r"#\w+", "", sentence)

    # hapus @
    sentence = sentence.replace('@', '')

    # hspus pic.twitter.com
    sentence = re.sub(r"(pic.twitter.com/[^\s]+)", '', sentence)

    # Hapus karakter non-alfanumerik
    # sentence = re.sub(r"[^\w\s]+", "", sentence)

    # Konversi ke representasi ASCII
    # sentence = unidecode(sentence)

    # Hapus spasi ekstra
    sentence = re.sub(r"\s+", " ", sentence)

    # Trim kalimat
    sentence = sentence.strip()

    # Tambahkan kalimat baru ke dalam list
    tweet_cleaned.append(sentence)

display(tweet[0])
print('-'*50)
display(tweet_cleaned[0])

'Kata @prabowo Indonesia tidak dihargai bangsa asing!   Berita ini ðŸ‘‡ pasti hoax buatan penguasa, ya kan @rockygerung?ðŸ˜œ https://twitter.com/mediaindonesia/status/1117575436337160192?s=21Â\xa0â€¦'

--------------------------------------------------


'Kata prabowo Indonesia tidak dihargai bangsa asing! Berita ini ðŸ‘‡ pasti hoax buatan penguasa, ya kan rockygerung?ðŸ˜œ https://twitter.com/mediaindonesia/status/1117575436337160192?s=21Â â€¦'

### Remove URL LINK

In [138]:
pattern = r"(https?://.[^\s]+)"
tweet_no_url = [re.sub(pattern, "", sentence) for sentence in tweet_cleaned]

display(tweet_cleaned[0])
print('-'*50)
display(tweet_no_url[0])

'Kata prabowo Indonesia tidak dihargai bangsa asing! Berita ini ðŸ‘‡ pasti hoax buatan penguasa, ya kan rockygerung?ðŸ˜œ https://twitter.com/mediaindonesia/status/1117575436337160192?s=21Â â€¦'

--------------------------------------------------


'Kata prabowo Indonesia tidak dihargai bangsa asing! Berita ini ðŸ‘‡ pasti hoax buatan penguasa, ya kan rockygerung?ðŸ˜œ  â€¦'

### Remove Punctuation & Special Character

In [139]:
def remove_punctuation(sentences):
  """Menghilangkan punctuation dari list kalimat."""

  # Buat regular expression untuk punctuation
  regex = re.compile(r"[^a-zA-Z0-9]")

  # Hapus punctuation dari setiap kalimat
  for sentence in sentences:
    sentence = regex.sub("", sentence)

  return sentences

def remove_punctuation_and_special_character(text):
  # clean puntuation and special character using regex
  cleaned_text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
  return cleaned_text

In [140]:
# regex = re.compile(r"[^a-zA-Z0-9]")

tweet_no_punct = []
for idx in range(len(tweet_no_url)):
  cleaned = remove_punctuation_and_special_character(tweet_no_url[idx])
  tweet_no_punct.append(cleaned)

print(tweet_no_url[0])
print(tweet_no_punct[0])


Kata prabowo Indonesia tidak dihargai bangsa asing! Berita ini ðŸ‘‡ pasti hoax buatan penguasa, ya kan rockygerung?ðŸ˜œ  â€¦
Kata prabowo Indonesia tidak dihargai bangsa asing Berita ini  pasti hoax buatan penguasa ya kan rockygerung  


### Remove Double Space

In [141]:
tweet_cleaned_double_space = []
def remove_double_space(text):
    cleaned_text = re.sub(r"\s+", " ", text)
    return cleaned_text

for index in range(len(tweet_no_punct)):
    text = remove_double_space(tweet_no_punct[index])
    tweet_cleaned_double_space.append(text)

print(tweet_no_punct[0])
print(tweet_cleaned_double_space[0])

Kata prabowo Indonesia tidak dihargai bangsa asing Berita ini  pasti hoax buatan penguasa ya kan rockygerung  
Kata prabowo Indonesia tidak dihargai bangsa asing Berita ini pasti hoax buatan penguasa ya kan rockygerung 


### Remove Stopwords

In [142]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
stop_factory = StopWordRemoverFactory()
# more_stopword = [‘dengan’, ‘ia’,’bahwa’,’oleh’]
# data = stop_factory.get_stop_words()+more_stopword
data = stop_factory.get_stop_words()
stopword_remover = stop_factory.create_stop_word_remover()
print(data)

['yang', 'untuk', 'pada', 'ke', 'para', 'namun', 'menurut', 'antara', 'dia', 'dua', 'ia', 'seperti', 'jika', 'jika', 'sehingga', 'kembali', 'dan', 'tidak', 'ini', 'karena', 'kepada', 'oleh', 'saat', 'harus', 'sementara', 'setelah', 'belum', 'kami', 'sekitar', 'bagi', 'serta', 'di', 'dari', 'telah', 'sebagai', 'masih', 'hal', 'ketika', 'adalah', 'itu', 'dalam', 'bisa', 'bahwa', 'atau', 'hanya', 'kita', 'dengan', 'akan', 'juga', 'ada', 'mereka', 'sudah', 'saya', 'terhadap', 'secara', 'agar', 'lain', 'anda', 'begitu', 'mengapa', 'kenapa', 'yaitu', 'yakni', 'daripada', 'itulah', 'lagi', 'maka', 'tentang', 'demi', 'dimana', 'kemana', 'pula', 'sambil', 'sebelum', 'sesudah', 'supaya', 'guna', 'kah', 'pun', 'sampai', 'sedangkan', 'selagi', 'sementara', 'tetapi', 'apakah', 'kecuali', 'sebab', 'selain', 'seolah', 'seraya', 'seterusnya', 'tanpa', 'agak', 'boleh', 'dapat', 'dsb', 'dst', 'dll', 'dahulu', 'dulunya', 'anu', 'demikian', 'tapi', 'ingin', 'juga', 'nggak', 'mari', 'nanti', 'melainkan', '

In [143]:
raw_stopwords = ['yang', 'untuk', 'pada', 'ke', 'para', 'namun', 'menurut', 'antara', 'dia', 'dua', 'ia', 'seperti', 'jika', 'jika', 'sehingga', 'kembali', 'dan', 'ini', 'karena', 'kepada', 'oleh', 'saat', 'harus', 'sementara', 'setelah', 'belum', 'kami', 'sekitar', 'bagi', 'serta', 'di', 'dari', 'telah', 'sebagai', 'masih', 'hal', 'ketika', 'adalah', 'itu', 'dalam', 'bisa', 'bahwa', 'atau', 'hanya', 'kita', 'dengan', 'akan', 'juga', 'ada', 'mereka', 'sudah', 'saya', 'terhadap', 'secara', 'agar', 'lain', 'anda', 'begitu', 'mengapa', 'kenapa', 'yaitu', 'yakni', 'daripada', 'itulah', 'lagi', 'maka', 'tentang', 'demi', 'dimana', 'kemana', 'pula', 'sambil', 'sebelum', 'sesudah', 'supaya', 'guna', 'kah', 'pun', 'sampai', 'sedangkan', 'selagi', 'sementara', 'tetapi', 'apakah', 'kecuali', 'sebab', 'selain', 'seolah', 'seraya', 'seterusnya', 'tanpa', 'agak', 'boleh', 'dapat', 'dsb', 'dst', 'dll', 'dahulu', 'dulunya', 'anu', 'demikian', 'tapi', 'ingin', 'juga', 'nggak', 'mari', 'nanti', 'melainkan', 'oh', 'ok', 'seharusnya', 'sebetulnya', 'setiap', 'setidaknya', 'sesuatu', 'pasti', 'saja', 'toh', 'walau', 'tolong', 'tentu', 'amat', 'apalagi', 'bagaimanapun']

In [144]:
def remove_stopwords(text):
    # Kode untuk menghapus stopwords
    stop_words = set(stopwords.words('indonesian'))
    tokens = word_tokenize(text)
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    cleaned_text = ' '.join(filtered_tokens)
    return cleaned_text

def remove_stopwords_sastrawi(text):
    # Kode untuk menghapus stopwords
    text_lower = text.lower()
    stop_words = set(raw_stopwords)
    tokens = word_tokenize(text_lower)
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    cleaned_text = ' '.join(filtered_tokens)
    return cleaned_text

In [145]:
tweet_no_stopwords = []
for index in range(len(tweet_cleaned_double_space)):
    cleaned_text = remove_stopwords_sastrawi(tweet_cleaned_double_space[index])
    tweet_no_stopwords.append(cleaned_text)

display(tweet_cleaned_double_space[0])
display(tweet_no_stopwords[0])

'Kata prabowo Indonesia tidak dihargai bangsa asing Berita ini pasti hoax buatan penguasa ya kan rockygerung '

'kata prabowo indonesia tidak dihargai bangsa asing berita hoax buatan penguasa ya kan rockygerung'

In [146]:
tweet_no_stopwords

['kata prabowo indonesia tidak dihargai bangsa asing berita hoax buatan penguasa ya kan rockygerung',
 'batuan langka tasbih jokowi hadiah habib luthfi seharga mercy',
 'era jokowi ekonomi indonesia semakin baik',
 'sumatera selatan asian games berdampak pd ekonomi langsung diprediksi mencapai 185 triliun indonesia maju jokowi hebat',
 'negara ngutang buat bngun infrastruktur udah dipake masyarakat terus masyarakatnya ngeluh negara ngutang negara ngutang utang dibayar kalo negara dapet penghasilan penghasilan negara ya pajak',
 'yg bisikin pak jokowi cm mikirin perputaran duit golongan tertentu esports sebagian kecil dr sekian besar penentu pertumbuhan perekonomian indonesia bukan fundamental betul yg dikatakan prabowo dgn bhs yg sederhana tp mrpkan aspek fundamental ekonomi',
 'masa tenang msih ngoceh ajattp jokowi harga mati',
 'kerjasa ekonomi bilateral multilateral doakan tps dengarkan suara hati',
 'iya bener aa mantap kan pilihan pemimpin yg bs memberi solusi jokowi bukan yg meny

### Replace Slang

In [159]:
# def replace_slang(text):
#     slang_model = spacy.load("id_core_web_lg")
#     slang_doc = slang_model(text)
#     slang_words = [token for token in slang_doc if token.pos == "ADJ"]
# def replace_slang_with_spacy(text, slang_dict):
#     nlp = spacy.load("id_core_web_md")
#     doc = nlp(text)
#     replaced_tokens = []

#     for token in doc:
#         if token.text in slang_dict:
#             replaced_tokens.append(slang_dict[token.text])
#         else:
#             replaced_tokens.append(token.text)

#     return " ".join(replaced_tokens)


def replace_slang_word(doc,slang_word):
    for index in  range(0,len(doc)-1):
        index_slang = slang_word.slang==doc[index]
        formal = list(set(slang_word[index_slang].formal))
        # if len(formal)==1:
        #     doc[index]=formal[0]
    return doc



In [156]:
indo_slang_dict = pd.read_csv("https://raw.githubusercontent.com/nasalsabila/kamus-alay/master/colloquial-indonesian-lexicon.csv")
indo_slang_dict.head()

,slang,formal,In-dictionary,context,category1,category2,category3
0,woww,wow,1,wow,elongasi,0,0
1,aminn,amin,1,Selamat ulang tahun kakak tulus semoga panjang...,elongasi,0,0
2,met,selamat,1,Met hari netaas kak!? Wish you all the best @t...,abreviasi,0,0
3,netaas,menetas,1,Met hari netaas kak!? Wish you all the best @t...,afiksasi,elongasi,0
4,keberpa,keberapa,0,Birthday yg keberpa kak?,abreviasi,0,0


In [160]:

# slang_dict.read()
tweet_no_slang = []
for index in range(len(tweet_no_stopwords)):
    new = replace_slang_word(tweet_no_stopwords[index], indo_slang_dict)
    tweet_no_slang.append(new)

display(tweet_no_stopwords[0])
displacy(tweet_no_slang[0])

KeyboardInterrupt: 

## Normalization

### Stemming

In [149]:
def normalization_stemming(text):
  # create an instance of the stemmer
  stemmer = StemmerFactory().create_stemmer()

  # perform stemming
#   cleaned_text = stemmer.stem(text)
  cleaned_text = stemmer.stem(text)
  return cleaned_text

In [150]:
normalized_stemming = []
for idx in range(len(tweet_no_stopwords)):
    normalized = normalization_stemming(tweet_no_stopwords[idx])
    normalized_stemming.append(normalized)

display(tweet_no_stopwords[1])
display(normalized_stemming[1])

KeyboardInterrupt: 

In [151]:
def normalization_lemmatizing(text):
  lemmatizer = WordNetLemmatizer()
  lemmatized = lemmatizer.lemmatize(text)
  return lemmatized

In [152]:
# normalized_lemmatizing = []
# for idx in range(len(tweet_no_stopwords)):
#     new = normalization_lemmatizing(tweet_no_stopwords[idx])
#     normalized_lemmatizing.append(new)

# display(tweet_no_stopwords[1])
# display(normalized_lemmatizing[1])

'batuan langka tasbih jokowi hadiah habib luthfi seharga mercy'

'batuan langka tasbih jokowi hadiah habib luthfi seharga mercy'